# Makam Project

Import Music21

In [2]:
from music21 import *

This cell will remove the key signature from every song in the SymbTr-2.4.3 directory. Note the path for the SymbTr directory and that it should be in the parent directory of the location of this notebook

In [1]:
import os
import xml.etree.ElementTree as ET

makamScoreDirectory = '../SymbTr-2.4.3/MusicXML/' ### Dataset should be at the parent directory
for dirPaths, dirNames, makamScores in os.walk(makamScoreDirectory):
    for makamScore in makamScores:
        tree = ET.parse(os.path.join(makamScoreDirectory, makamScore))
        root = tree.getroot()
        for att in root.iter('attributes'):
            if att.find('key'):
                att.remove(att.find('key'))
        tree.write(os.path.join(makamScoreDirectory, makamScore))

Extract Pitch classes from files

In [22]:
makams = {} # object to hold all the makam types with pitch distributions
usuls = {} # object to hold all the usul types with pitch distributions

# Iterate over all files in the dataset
for dirPaths, dirNames, makamScores in os.walk(makamScoreDirectory):
    for file in makamScores:
        # Parse file name to get makam and usul type
        fileNameSegments = file.split('--')
        makamType = fileNameSegments[0]
        usulType = fileNameSegments[2]
        
        makamScore = os.path.join(makamScoreDirectory, file)
        score = converter.parse(makamScore)
        allNotes = score.flat.notes.stream()
        
        results = [] # array of pitches to store the results
        for note in allNotes:
            result = note.pitch.step
            if note.pitch.accidental:
                result += '_' + note.pitch.accidental.name
            results.append(result)
            
        if makamType not in makams:
            makams[makamType] = results
        else:
            makams[makamType] += results
        
        if usulType not in usuls:
            usuls[usulType] = results
        else:
            usuls[usulType] += results

In [23]:
import json

with open('./makam-distribution.json', "w") as fp:
    json.dump(makams, fp)
with open('./usul-distribution.json', "w") as fp:
    json.dump(usuls, fp)